In [3]:
# try the rfc us model on the dev train,  and then test dev test data --- down
# try new rfc model , under sampled, hyperparameter tuned on train data, and then test the dev test data

In [2]:
import pandas as pd
import polars as pl
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
# import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
# import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA
import joblib

import spacy
from collections import Counter
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from pprint import pprint

In [4]:
dev_test = pd.read_csv('created_new_dev_data/first_dev_test.csv')
dev_train = pd.read_csv('created_new_dev_data/first_dev_train.csv')

In [7]:
dev_train.columns

Index(['Sentence_id', 'Text', 'class_label', 'Tokens', 'text_length',
       'cleaned_text', 'cleaned_text_length', 'roberta_sentiment',
       'roberta_sent_pos', 'roberta_sent_neg', 'roberta_sent_mixed',
       'roberta_sentiment_class', 'count_tokens', 'counts', 'names',
       'organizations', 'dates', 'punctuation_count', 'flattened_embedding',
       'label'],
      dtype='object')

In [6]:
dev_train.drop(['converted_embedding', 'embedding'], axis=1, inplace = True)

In [8]:
dev_train.dtypes

Sentence_id                  int64
Text                        object
class_label                 object
Tokens                      object
text_length                  int64
cleaned_text                object
cleaned_text_length          int64
roberta_sentiment           object
roberta_sent_pos           float64
roberta_sent_neg           float64
roberta_sent_mixed         float64
roberta_sentiment_class     object
count_tokens                 int64
counts                      object
names                        int64
organizations                int64
dates                        int64
punctuation_count            int64
flattened_embedding         object
label                        int64
dtype: object

In [18]:
# dev_train['flattened_embedding'].iloc[0]
# convert to numpy array
def convert_to_numpy_array(embd_str):
     #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [19]:
dev_train['converted_embedding'] = dev_train['flattened_embedding'].apply(convert_to_numpy_array)

In [21]:
dev_train['converted_embedding'].iloc[0].shape

(768,)

In [9]:
dev_train = dev_train.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [22]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(dev_train['converted_embedding'].to_list())

In [24]:
text = dev_train['Text']
vectorizer_path = 'models/tfidf_vectorizer_10000.pkl'
tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
tfidf_features = tfidf_vect.transform(text)

In [26]:
additional_features = dev_train[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [29]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [33]:
X_train = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])
y_train = dev_train['label']

In [34]:
# loading the best model so far
# load model rfc us
# trained on original train data (not dev data)
model_path_rfc_us = 'models/random_forest_tfidf_embeddings_10_features_us.pkl'
model_rfc_us = joblib.load(model_path_rfc_us)
# predictions_rfc_us = model_rfc_us.predict(X)

In [35]:
model_rfc_us.fit(X_train,y_train)

RandomForestClassifier()

In [36]:
# test the data on dev_test

In [37]:
dev_test.sample(1)

,Sentence_id,Text,class_label,label,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,count_tokens,...,roberta_sent_neg,roberta_sent_mixed,roberta_sentiment_class,names,organizations,dates,punctuation_count,embedding,converted_embedding,flattened_embedding
249,40263,And the idea that we're in a situation and are...,Yes,1,"['idea', 'situation', 'going', 'destroy', 'Med...",324,idea situation going destroy medicare guy actu...,181,"{'positive_score': 0.014764818362891674, 'nega...",26,...,0.788569,0.196666,Negative,0,4,1,0,[[ 2.22092360e-01 4.56784815e-01 -1.82840645e...,[[ 2.22092360e-01 4.56784815e-01 -1.82840645e...,[ 2.22092360e-01 4.56784815e-01 -1.82840645e-...


In [39]:
# dev_test['flattened_embedding'].iloc[0]
dev_test['converted_embedding'] = dev_test['flattened_embedding'].apply(convert_to_numpy_array)

In [40]:
dev_test = dev_test.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [43]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(dev_test['converted_embedding'].to_list())

In [44]:
text = dev_test['Text']
vectorizer_path = 'models/tfidf_vectorizer_10000.pkl'
tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
tfidf_features = tfidf_vect.transform(text)

In [49]:
additional_features = dev_test[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [50]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [51]:
X_test = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])
y_test = dev_train['label']

In [54]:
y_pred = model_rfc_us.predict(X_test)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]


In [68]:
test_dataframe = pd.DataFrame({
    'id': dev_test['Sentence_id'],
    'predictions': y_pred
})
new_df = dev_test[['Sentence_id', 'class_label']].copy()
test_dataframe['predictions'] = test_dataframe['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
df_merged = pd.merge(new_df, test_dataframe, left_on='Sentence_id', right_on='id', how='inner')

original_labels = df_merged['class_label'].tolist()
pred_labels = df_merged['predictions'].tolist()

print(f"RFC US")
acc = accuracy_score(original_labels, pred_labels)
precision = precision_score(original_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(original_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(original_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

RFC US
Accuracy: 0.6823899371069182
Precision: 0.8888888888888888
Recall: 0.07407407407407407
F1 Score: 0.13675213675213677
